In [1]:
pip install -r requirements.txt

     -------------------------------------- 331.8/331.8 MB 5.8 MB/s eta 0:00:00
  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl (13.1 MB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl (7.0 MB)
     ---------------------------------------- 8.1/8.1 MB 6.4 MB/s eta 0:00:00
  Using cached streamlit-1.51.0-py3-none-any.whl (10.2 MB)
     -------------------------------------- 135.8/135.8 kB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 26.4/26.4 MB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
  Using cached protobuf-6.33.1-cp310-abi3-win_amd64.whl (436 kB)
  Using cached requests-2.32.5-py3-none-any.whl (64 kB)
     ---------------------------------------- 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 4.7/4.7 MB 8.8 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 6.4 MB/


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# -----------------------
# CONFIG
# -----------------------
DATA_DIR = "data"          
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 5                 
MODEL_PATH = "models/ai_detector_model.h5"

os.makedirs("models", exist_ok=True)

# -----------------------
# DATA: AUTO TRAIN/VAL SPLIT 
# -----------------------
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",     #
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"    
)

print("Class indices:", train_gen.class_indices)


# -----------------------
# SIMPLE CNN MODEL
# -----------------------
def build_model(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Last conv block 
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="last_conv")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Classifier
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)  # binary: ai vs real

    model = models.Model(inputs, outputs)
    return model

model = build_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -----------------------
# TRAIN
# -----------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

# -----------------------
# SAVE MODEL
# -----------------------
model.save(MODEL_PATH)
print(f"[INFO] Model saved to {MODEL_PATH}")


Found 777 images belonging to 2 classes.
Found 193 images belonging to 2 classes.
Class indices: {'ai': 0, 'real': 1}


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ last_conv (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     6,422,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,515,905 (24.86 MB)

 Trainable params: 6,515,905 (24.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
18/49 ━━━━━━━━━━━━━━━━━━━━ 17s 565ms/step - accuracy: 0.5963 - loss: 1.1776

c:\Users\aqilw\Desktop\project\AI-Detector\.venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


49/49 ━━━━━━━━━━━━━━━━━━━━ 34s 681ms/step - accuracy: 0.5727 - loss: 0.7669 - val_accuracy: 0.5544 - val_loss: 0.6535
Epoch 2/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 24s 487ms/step - accuracy: 0.6165 - loss: 0.6683 - val_accuracy: 0.5803 - val_loss: 0.6783
Epoch 3/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 23s 470ms/step - accuracy: 0.6100 - loss: 0.6459 - val_accuracy: 0.6788 - val_loss: 0.6286
Epoch 4/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 23s 478ms/step - accuracy: 0.7001 - loss: 0.5797 - val_accuracy: 0.6425 - val_loss: 0.6203
Epoch 5/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 23s 463ms/step - accuracy: 0.7722 - loss: 0.4804 - val_accuracy: 0.6632 - val_loss: 0.6957


[INFO] Model saved to models/ai_detector_model.h5
